In [1]:
import pandas as pd

%load_ext autoreload
%autoreload 2

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper'.format(folder)
bed_folder = '{}/bed'.format(folder)
# folder2 = '/projects/ps-yeolab2/obotvinnik/singlecell_pnm'

splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

alt_exons_bedfile = '{}/exons.bed'.format(alternative_feature_folder)
constitutive_bedfile = '{}/exons.bed'.format(constitutive_feature_folder)


import pybedtools

alt_exons = pybedtools.BedTool(alt_exons_bedfile)
constitutive = pybedtools.BedTool(constitutive_bedfile)

from Bio import SeqIO
import kvector
import pybedtools
import pyhomer

DIRECTIONS = 'upstream', 'downstream'

placental_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed'
conservation_bed = pybedtools.BedTool(placental_filename)

genome = 'hg19'

nt = 200

genome_fasta = '/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa'

pair = pyhomer.ForegroundBackgroundPair(alt_exons, constitutive)


In [2]:
alternative_feature_folder

'/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative'

In [3]:
constitutive_feature_folder

'/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/constitutive'

In [4]:
ls $alternative_feature_folder/*.bed

/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exon2_phastcons_placental_mammal.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_downstream200nt.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_merkin2012_ancient_alt_exons.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_phastcons_placental_mammal.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_stop_plus1.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_upstream200_foreground.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_upstream200_placental_f

In [10]:
! wc -l $alternative_feature_folder/*.bed

   26379 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exon2_phastcons_placental_mammal.bed
   34982 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons.bed
   43903 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_downstream200nt.bed
     177 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_merkin2012_ancient_alt_exons.bed
   43903 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_phastcons_placental_mammal.bed
   43903 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_stop_plus1.bed
   43903 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_upstream200_foreground.bed
   53712 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_f

In [5]:
! head $alt_exons_bedfile > $alternative_feature_folder/test.bed

Cacluate the time needed for 10 intervals

In [6]:
%%time

import kvector

kmers = kvector.kmer.per_interval_kmers('{}/test.bed'.format(alternative_feature_folder), genome_fasta, conservation_bed)

CPU times: user 178 ms, sys: 109 ms, total: 287 ms
Wall time: 35.8 s


Total time per 10 exons

```
CPU times: user 178 ms, sys: 109 ms, total: 287 ms
Wall time: 35.8 s
```


In [20]:
cd ~/projects/singlecell_pnms/scripts/

/home/obotvinnik/processing_scripts/singlecell_pnms


In [21]:
%%file ~/count_interval_kmers.py

import sys
import kvector

bedfile = sys.argv[1]
genome_fasta = sys.argv[2]
other = sys.argv[3]
threads = int(sys.argv[4])

kmers = kvector.kmer.per_interval_kmers(bedfile, genome_fasta, other, threads=threads)
kmers = kmers.astype(int)
csv = bedfile.replace('.bed', '_kmers.csv')
kmers.to_csv(csv)

Overwriting /home/obotvinnik/count_interval_kmers.py


In [32]:
n_threads = 8

In [35]:
n_lines = ! wc -l $alt_exons_bedfile

multipler = int(n_lines[0].split()[0])/float(n_threads)
multipler

4372.75

35.8s for 10 exons -- divide by 60 twice to get the number of hours

In [36]:
(multipler * 35.8)/60/60

43.48456944444444

what about for constitutive exons?

In [34]:
n_lines = ! wc -l $constitutive_bedfile
multipler = int(n_lines[0].split()[0])/float(n_threads)
multipler

2192.625

Even less -- so 48 hours will work for both

In [37]:
import qtools

def kmer_command(bedfile, genome_fasta, other, threads):
    return 'time python ~/count_interval_kmers.py {} {} {} {}'.format(bedfile, genome_fasta, other, threads)

bedfiles = alt_exons_bedfile, constitutive_bedfile
threads = 8

for bedfile in bedfiles:
    jobname = bedfile.split('/')[-2] + '_kmers'
    
    commands = []
    
    n_lines = ! wc -l $bedfile
    n_lines = int(n_lines[0].split()[0])
    n_minutes = 4 * n_lines / 10
    
    commands.append(kmer_command(bedfile, genome_fasta, placental_filename, threads))
    
    for direction in DIRECTIONS:
        bed = pybedtools.BedTool(bedfile)
        if direction == 'downstream':
            intron = bed.flank(l=0, r=nt, s=True, genome=genome)
        elif direction == 'upstream':
            intron = bed.flank(l=nt, r=0, s=True, genome=genome)
        intron_filename = bedfile.replace('.bed', '_{}{}nt.bed'.format(direction, nt))
        intron.saveas(intron_filename)
        commands.append(kmer_command(intron_filename, genome_fasta, placental_filename, threads))
#     print commands
    qtools.Submitter(commands, jobname, walltime='48:00:00'.format(n_minutes), array=True, ppn=threads)

Writing 3 tasks as an array-job.
Wrote commands to alternative_kmers.sh.
Submitted script to queue home.
 Job ID: 7388244
Writing 3 tasks as an array-job.
Wrote commands to constitutive_kmers.sh.
Submitted script to queue home.
 Job ID: 7388245


In [38]:
ls *kmers*

alternative_kmers.sh  constitutive_kmers.sh


In [39]:
! tail *kmers*.out*

tail: cannot open `*kmers*.out*' for reading: No such file or directory


In [41]:
! ls -lh $alternative_feature_folder/*kmers*

-rw-r--r-- 1 obotvinnik yeo-group 455M May 21  2016 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_downstream200nt_kmers.csv
-rw-r--r-- 1 obotvinnik yeo-group 455M May 21  2016 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_kmers.csv
-rw-r--r-- 1 obotvinnik yeo-group 455M May 21  2016 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_upstream200nt_kmers.csv


In [9]:
! ls $constitutive_feature_folder/*kmers*

/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/constitutive/exons_downstream200nt_kmers.csv
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/constitutive/exons_kmers.csv
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/constitutive/exons_upstream200nt_kmers.csv


In [3]:
! tail *kmers*.err*

==> alternative_kmers.sh.err-1 <==
/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())

==> alternative_kmers.sh.err-2 <==
/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())

==> alternative_kmers.sh.err-3 <==
/